In [1]:
#Libraries 

import pandas as pd
import numpy as np

# Text Processing:

import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import FrenchStemmer

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Cansu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# Chargement des données

X_train = pd.read_csv("../data/X_train_update.csv", index_col = 0)
y_train = pd.read_csv("../data/Y_train_CVw08PX.csv", index_col = 0)
X_test = pd.read_csv("../data/X_test_update.csv", index_col = 0)

In [3]:
# Stopwords

html_stopwords = ['br', 'p', 'div', 'span', 'b', 'i', 'li', 'ul', 'strong', 'em']
new_stop_words = [",", ".", "``", "@", "*", "(", ")", "...", "!", "?", "-", 
                  "_", ">", "<", ":", "/", "=", "--", "©", "~", ";", "\\", "\\\\"]
final_stopwords = stopwords.words('english') + stopwords.words('french') + html_stopwords + new_stop_words


In [4]:
# Stemming and processing

stemmer = FrenchStemmer()

def stemming(mots) :
    sortie = []
    for string in mots :
        radical = stemmer.stem(string)
        if (radical not in sortie) : sortie.append(radical)
    return sortie


def preprocessing(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_removed = [word for word in tokens if word not in final_stopwords]
    stemmed = stemming(stop_removed)
    return ' '.join(stemmed)

    

In [5]:
# Stemmed variable:

X_train['preprocessed_designation'] = ""

for i in range(0, len(X_train)):
    X_train.loc[i, 'preprocessed_designation'] = preprocessing(X_train.loc[i, 'designation'])

In [8]:
X_train[["designation", "preprocessed_designation"]].head(10)

,designation,preprocessed_designation
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,olivi personalisiert notizbuch 150 seiten punk...
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,journal art n° 133 28/09/2001 l'art march salo...
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,grand stylet ergonom bleu gamepad nintendo wii...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,peluch donald europ disneyland 2000 marionnet ...
4,La Guerre Des Tuques,guerr tuqu
5,Afrique Contemporaine N° 212 Hiver 2004 - Doss...,afriqu contemporain n° 212 hiv 2004 dossi japon
6,Christof E: Bildungsprozessen Auf Der Spur,christof e bildungsprozessen auf der spur
7,Conquérant Sept Cahier Couverture Polypro 240 ...,conquer sept cahi couvertur polypro 240 x 320 ...
8,Puzzle Scooby-Doo Avec Poster 2x35 Pieces,puzzl scooby-doo post 2x35 piec
9,Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - ...,tent pli v3s5-pro pvc blanc 3 x 4m50 longueur ...
